In [59]:
import json
import os
import glob
import pandas as pd
import requests
import time
from datetime import datetime, timedelta
from random import randint
from pga.return_pid import return_pid
from sqlalchemy import create_engine
from pandas.io import sql
from pandas.io.json import json_normalize

In [31]:
##Retrieve All Weeks data from fantasylabs

os.chdir('/home/valesco/Datasets/NFL/nfl_data/salaries')

url = 'http://www.fantasylabs.com/api/playermodel/1/{}/?modelId=1196314&projOnly=true'
headers = { 'User-agent': 'Mozilla/5.0',
            'Cookie':'__cfduid=d2e9d0d941e292062b5792a57cd6183f11505095681; LD_U=http%3A%2F%2Fwww.fantasylabs.com%2F; __distillery=ddea8bf_c6fa751c-2767-47ec-9073-7092b00d479d-0862dfd0d-8888296cfa38-33df; LD_R=http%3A%2F%2Fwww.fantasylabs.com%2Fnfl%2Fcontests-dashboard%2F; flid=V2y0f5FbyE2AwjnR1hOc8Q; .AspNet.Cookies=xdR0OEb0eFLJHQUN8AHj7GUgPf9UwvV7Ks9sl3A3_kbN3yck1UqXrGiRI_5RyYd4lkgxsVMs5YASqBFadrPTiFqFLxdpfSfbNexcrtFT7SG61YIK1AVcBm0HRF4PwBkE5BVnR4K5f912gATMkqWj0cSu8P6DpprZ4WKaGDz25Ae58aaYso0VEQXvtu_Fauchk0ljT536JdwRMq3y247kRCcVFrqXY5Evjp7h28uTTckQVvQTX7Tqf8NfN_wviZHOTRDEYvpxxUXlo7ElV0_ulB4oVQUBK3QDFi98W6THNqjJiDiQvFz9TAfpScsS5G9ZQ3axBjOzJqufqjLq17mLWmr3iwnamPeb3VsaUSLYi5k2ivJCUxRqrKQBY-nQLyXunZ0W2BHe4Wr8Xe-OLquugIZPui38kAerfotnY5-e3qPYgZ2LsQzofY7mMGpMpelmovf2cuyZ8iDyeooJo25wtA; __zlcmid=iSgjnvnLj8Z3xC; _ga=GA1.2.993633943.1505095683; _gid=GA1.2.1894999805.1509310833; LD_T=8265517e-bdb6-44e0-fd38-0f5bcbdfb4a5; hasmembership=True'
           }


event_ls = []

now = datetime.now()

for year in ['2014', '2015', '2016', '2017']:
    for month in ['1', '2', '9', '10', '11', '12']:
        for day in range(1,31):
            date_str = '{}_{}_{}'.format(month, day, year)

            complete_url = 'http://www.fantasylabs.com/api/playermodel/1/{}/?modelId=1196314&projOnly=true'
            
            time.sleep(.3)
            
            data = requests.get(url.format(date_str), headers = headers).json()
            temp_event_ls = []
            
            for d in data['PlayerModels']:
                if d['Properties']['EventId'] not in event_ls:
                    temp_event_ls.append(d['Properties']['EventId'])
                    
            count = 0

            for event in temp_event_ls:
                if event not in event_ls:
                    event_ls.append(event)
                    count += 1

            if count > 1:
                file_name = 'nfl' + date_str + '.json'

                with open(file_name, 'w') as outfile:
                    json.dump(data, outfile)

                print(date_str + ' file saved')


            



1_1_2016 file saved
1_1_2017 file saved
1_3_2017 file saved
1_10_2017 file saved
1_17_2017 file saved


In [94]:
## Parse all historical files into csv

files = glob.glob('*')
main_df = pd.DataFrame()


for file in files:
    date_str = datetime.strptime(file.replace('nfl', '').replace('.json', ''), '%m_%d_%Y')
    
    with open(file) as json_file:
        data = json.load(json_file)

    df = json_normalize(data['PlayerModels'])
        
    df['date'] = date_str
        
    main_df = main_df.append(df, ignore_index = True)
        
        
main_df = main_df[main_df['Properties.SourceId'] == 4]
main_df['date'] = main_df['date'].apply(lambda x: pd.to_datetime(x))
main_df.sort_values('date', inplace = True) 
main_df.reset_index(inplace = True, drop = True)
main_df.to_csv('nfl_data.csv', index = False)

(52914, 164)

In [124]:
#Add weeks 1 by 1 to dataframe
main_df = pd.read_csv('nfl_data.csv')

event_ls = list(main_df['Properties.EventId'].unique())

now = datetime.now()

os.chdir('/home/valesco/Datasets/NFL/nfl_data/salaries')

url = 'http://www.fantasylabs.com/api/playermodel/1/{}/?modelId=1196314&projOnly=true'
headers = { 'User-agent': 'Mozilla/5.0',
            'Cookie':'__cfduid=d2e9d0d941e292062b5792a57cd6183f11505095681; LD_U=http%3A%2F%2Fwww.fantasylabs.com%2F; __distillery=ddea8bf_c6fa751c-2767-47ec-9073-7092b00d479d-0862dfd0d-8888296cfa38-33df; LD_R=http%3A%2F%2Fwww.fantasylabs.com%2Fnfl%2Fcontests-dashboard%2F; flid=V2y0f5FbyE2AwjnR1hOc8Q; .AspNet.Cookies=xdR0OEb0eFLJHQUN8AHj7GUgPf9UwvV7Ks9sl3A3_kbN3yck1UqXrGiRI_5RyYd4lkgxsVMs5YASqBFadrPTiFqFLxdpfSfbNexcrtFT7SG61YIK1AVcBm0HRF4PwBkE5BVnR4K5f912gATMkqWj0cSu8P6DpprZ4WKaGDz25Ae58aaYso0VEQXvtu_Fauchk0ljT536JdwRMq3y247kRCcVFrqXY5Evjp7h28uTTckQVvQTX7Tqf8NfN_wviZHOTRDEYvpxxUXlo7ElV0_ulB4oVQUBK3QDFi98W6THNqjJiDiQvFz9TAfpScsS5G9ZQ3axBjOzJqufqjLq17mLWmr3iwnamPeb3VsaUSLYi5k2ivJCUxRqrKQBY-nQLyXunZ0W2BHe4Wr8Xe-OLquugIZPui38kAerfotnY5-e3qPYgZ2LsQzofY7mMGpMpelmovf2cuyZ8iDyeooJo25wtA; __zlcmid=iSgjnvnLj8Z3xC; _ga=GA1.2.993633943.1505095683; _gid=GA1.2.1894999805.1509310833; LD_T=8265517e-bdb6-44e0-fd38-0f5bcbdfb4a5; hasmembership=True'
           }

for i in range(7,0, -1):
    date_str = datetime.strftime(now - timedelta(days = 1), '%m_%d_%Y')
    
    time.sleep(.3)
            
    data = requests.get(url.format(date_str), headers = headers).json()
    temp_event_ls = []

    for d in data['PlayerModels']:
        if d['Properties']['EventId'] not in event_ls:
            temp_event_ls.append(d['Properties']['EventId'])
            
    count = 0

    for event in temp_event_ls:
        if event not in event_ls:
            event_ls.append(event)
            count += 1

    if count > 1:
        file_name = 'nfl' + date_str + '.json'

        with open(file_name, 'w') as outfile:
            json.dump(data, outfile)

        print(date_str + ' file saved')
        
        df = json_normalize(data['PlayerModels'])
        
        df['date'] = datetime.strptime(date_str, '%m_%d_%Y')
        
        df = df[df['Properties.SourceId'] == 4]

        main_df = main_df.append(df, ignore_index = True)

/home/valesco/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (159) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


10_30_2017 file saved


In [181]:
##Get current salaries for the week
def position_dummy(x):
    if x == 'QB':
        return 1
    if x == 'RB': 
        return 2
    if x == 'WR':
        return 3
    if x == 'TE':
        return 4
    if x == 'D':
        return 5


now_str = datetime.strftime(datetime.now(), '%m_%d_%Y')

url = 'http://www.fantasylabs.com/api/playermodel/1/{}/?modelId=1196314&projOnly=true'.format(now_str)

headers = { 'User-agent': 'Mozilla/5.0',
            'Cookie':'_cfduid=d2e9d0d941e292062b5792a57cd6183f11505095681; LD_U=http%3A%2F%2Fwww.fantasylabs.com%2F; __distillery=ddea8bf_c6fa751c-2767-47ec-9073-7092b00d479d-0862dfd0d-8888296cfa38-33df; LD_R=http%3A%2F%2Fwww.fantasylabs.com%2Fnfl%2Fcontests-dashboard%2F; flid=V2y0f5FbyE2AwjnR1hOc8Q; .AspNet.Cookies=xdR0OEb0eFLJHQUN8AHj7GUgPf9UwvV7Ks9sl3A3_kbN3yck1UqXrGiRI_5RyYd4lkgxsVMs5YASqBFadrPTiFqFLxdpfSfbNexcrtFT7SG61YIK1AVcBm0HRF4PwBkE5BVnR4K5f912gATMkqWj0cSu8P6DpprZ4WKaGDz25Ae58aaYso0VEQXvtu_Fauchk0ljT536JdwRMq3y247kRCcVFrqXY5Evjp7h28uTTckQVvQTX7Tqf8NfN_wviZHOTRDEYvpxxUXlo7ElV0_ulB4oVQUBK3QDFi98W6THNqjJiDiQvFz9TAfpScsS5G9ZQ3axBjOzJqufqjLq17mLWmr3iwnamPeb3VsaUSLYi5k2ivJCUxRqrKQBY-nQLyXunZ0W2BHe4Wr8Xe-OLquugIZPui38kAerfotnY5-e3qPYgZ2LsQzofY7mMGpMpelmovf2cuyZ8iDyeooJo25wtA; __zlcmid=iSgjnvnLj8Z3xC; _ga=GA1.2.993633943.1505095683; LD_T=8265517e-bdb6-44e0-fd38-0f5bcbdfb4a5; hasmembership=True'
           }


response = requests.get(url, headers = headers)
json_data = response.json()

week_all = json_normalize(json_data['PlayerModels'])

week_dk = week_all[['Properties.Player_Name', 'Properties.PlayerId', 'Properties.Position', 'Salary', 'Properties.Team', 'Properties.Opposing_TeamFB']].loc[week_all['Properties.SourceId'] == 4]
week_dk.columns = ['PlayerName', 'PlayerId', 'Position', 'Salary', 'Team', 'Opponent']
week_dk['PositionNum'] = week_dk['Position'].apply(lambda x: position_dummy(x))
dk_json = json.loads(week_dk.reset_index(drop = True).to_json(orient = 'records'))


In [182]:
dk_json

[{'Opponent': '@NYJ',
  'PlayerId': 3851,
  'PlayerName': 'Andre Holmes',
  'Position': 'WR',
  'PositionNum': 3,
  'Salary': 3100,
  'Team': 'BUF'},
 {'Opponent': '@NYJ',
  'PlayerId': 4987,
  'PlayerName': 'Jordan Matthews',
  'Position': 'WR',
  'PositionNum': 3,
  'Salary': 3900,
  'Team': 'BUF'},
 {'Opponent': '@NYJ',
  'PlayerId': 5069,
  'PlayerName': 'LeSean McCoy',
  'Position': 'RB',
  'PositionNum': 2,
  'Salary': 8700,
  'Team': 'BUF'},
 {'Opponent': '@NYJ',
  'PlayerId': 7174,
  'PlayerName': 'Tyrod Taylor',
  'Position': 'QB',
  'PositionNum': 1,
  'Salary': 5900,
  'Team': 'BUF'},
 {'Opponent': '@NYJ',
  'PlayerId': 7229,
  'PlayerName': 'Deonte Thompson',
  'Position': 'WR',
  'PositionNum': 3,
  'Salary': 3700,
  'Team': 'BUF'},
 {'Opponent': '@NYJ',
  'PlayerId': 7251,
  'PlayerName': 'Logan Thomas',
  'Position': 'TE',
  'PositionNum': 4,
  'Salary': 2600,
  'Team': 'BUF'},
 {'Opponent': '@NYJ',
  'PlayerId': 7297,
  'PlayerName': 'Mike Tolbert',
  'Position': 'RB',


In [175]:
for player in dk_json:
    temp_df = main_df[main_df['Properties.PlayerId'] == player['PlayerId']]
    temp_df.sort_values('date', inplace = True)
    
    distro = list(temp_df['ActualPoints'].values)
    
    player['Distro'] = distro
    
with open(now_str + '_nfl_processed.json', 'w') as outfile:
    json.dump(dk_json, outfile)

/home/valesco/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
